In [1]:
from PIL import Image, ImageFont, ImageDraw 
import glob
import string
import random
import os
import shutil
import math
import tensorflow as tf
print(tf.__version__)
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt 
from keras import backend as K 
import cv2
import random
import shutil
import stat

width = 128
height = 128

2.9.0


In [2]:
data_path = "C:/Users/Scrip0/Desktop/Internships/Magician AI/MagicianDatasetApp/Data"
ext = ["jpg","png", "jpeg"];

def read_img(path):
    img = cv2.resize(cv2.imread(path, cv2.IMREAD_COLOR), (width, height))
    return img

def equalize_dataset(path):
    c = 0
    true_dirs = []
    for it in os.scandir(path + "/False"):
        if it.is_dir():
            c += 1
    for it in os.scandir(path + "/True"):
        if it.is_dir():
            true_dirs.append(it.path)
    
    while (len(true_dirs) > c):
        dir = random.choice(true_dirs)
        true_dirs.remove(dir)
        os.chmod(dir, stat.S_IRUSR | stat.S_IWUSR | stat.S_IXUSR)
        shutil.rmtree(dir)

In [3]:
equalize_dataset(data_path)
imgs = []
labels = []

for path in glob.glob(f'{data_path + "/True"}/*/'):
    for files in (files for files in (glob.glob(path + '/*.%s' % e) for e in ext) if files != []):
        for file in files:
            imgs.append(read_img(file))
            labels.append(1.0)

for path in glob.glob(f'{data_path + "/False"}/*/'):
    for files in (files for files in (glob.glob(path + '/*.%s' % e) for e in ext) if files != []):
        for file in files:
            imgs.append(read_img(file))
            labels.append(0.0)
            
print("Imgs len: ", len(imgs))
print("Labels len: ", len(labels))

Imgs len:  600
Labels len:  600


In [4]:
imgs, labels = shuffle(imgs, labels)

In [5]:
imgs = np.array(imgs) / 256.0
labels = np.array(labels)

In [6]:
x_train, x_test, y_train, y_test = train_test_split(imgs, labels, train_size=0.8, random_state=1) 
images = []
labels = []

In [7]:
print(x_train.shape)
print(x_test.shape)

(480, 128, 128, 3)
(120, 128, 128, 3)


In [8]:
model = keras.Sequential()

model.add(keras.Input(shape=(width, height, 3))) #3 if RGB, 1 if GrayScale

model.add(tf.keras.layers.Conv2D(128, 5, activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(128, 5, activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(64, 5, activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(32, 5, activation="relu", padding="same"))
model.add(tf.keras.layers.MaxPooling2D(2))
model.add(tf.keras.layers.BatchNormalization())


model.add(Flatten())
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Dense(64, activation="relu"))

model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 128)     9728      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 128)      0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 128)      512       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 128)       409728    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 128)      0         
 2D)                                                             
                                                        

In [10]:
model.compile(optimizer=Adam(learning_rate=0.0005), loss="MeanSquaredError")

In [11]:
early_stop = EarlyStopping(patience=5, monitor='val_loss', restore_best_weights=True)
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=20, epochs=100, callbacks=[early_stop])

Epoch 1/100
24/24 [==============================] - 7s 141ms/step - loss: 0.3916 - val_loss: 0.2698
Epoch 2/100
24/24 [==============================] - 2s 81ms/step - loss: 0.2534 - val_loss: 0.2396
Epoch 3/100
24/24 [==============================] - 2s 80ms/step - loss: 0.2336 - val_loss: 0.2428
Epoch 4/100
24/24 [==============================] - 2s 80ms/step - loss: 0.2179 - val_loss: 0.2395
Epoch 5/100
24/24 [==============================] - 2s 80ms/step - loss: 0.2113 - val_loss: 0.2442
Epoch 6/100
24/24 [==============================] - 2s 80ms/step - loss: 0.2225 - val_loss: 0.2502
Epoch 7/100
24/24 [==============================] - 2s 80ms/step - loss: 0.1910 - val_loss: 0.2459
Epoch 8/100
24/24 [==============================] - 2s 80ms/step - loss: 0.1619 - val_loss: 0.2485
Epoch 9/100
24/24 [==============================] - 2s 81ms/step - loss: 0.1697 - val_loss: 0.2505


In [12]:
model.save('Face_recognition.h5')

In [13]:
result = model.predict(x_test).flatten()
print(result)
result = []
K.clear_session()

4/4 [==============================] - 3s 349ms/step
[0.56771976 0.5908613  0.5660716  0.5773761  0.5967905  0.5774151
 0.58186823 0.595079   0.5713429  0.5777708  0.5742147  0.5647354
 0.5639777  0.5764492  0.5917775  0.5772131  0.5846955  0.5849412
 0.5755203  0.5656903  0.58567023 0.5798804  0.5716051  0.59469867
 0.5888333  0.57973856 0.5798804  0.5716051  0.5790182  0.5770877
 0.5906457  0.58477014 0.5698273  0.5668689  0.596916   0.5763422
 0.5844596  0.59731597 0.57674396 0.584375   0.57146615 0.5768574
 0.59390384 0.569827   0.56980264 0.5821817  0.59228826 0.5765069
 0.5759804  0.591582   0.5711896  0.5957708  0.5659428  0.5952689
 0.57303566 0.5816515  0.57102877 0.58477014 0.5784501  0.5843603
 0.5958815  0.58160824 0.58399004 0.58296096 0.5664884  0.57044435
 0.5725015  0.5628801  0.58425575 0.5850159  0.57255036 0.57900906
 0.5723183  0.5748968  0.59231144 0.5922974  0.57390964 0.5786078
 0.57973856 0.57359844 0.5952491  0.5656601  0.5663473  0.59196836
 0.57905567 0.59493

In [14]:
x_train, x_test, y_train, y_test = [], [], [], []